# Nazwa modelu


## Opis


In [7]:
import os
import shutil
from pathlib import Path
import kaggle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image
import random
from torchvision.models import resnet18
from torchvision import transforms
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn


In [8]:
def load(tr_path):
    classes, class_paths = zip(*[(label, os.path.join(tr_path, label, image))
                                 for label in os.listdir(tr_path) if os.path.isdir(os.path.join(tr_path, label))
                                 for image in os.listdir(os.path.join(tr_path, label))])

    return pd.DataFrame({'Class Path': class_paths, 'Class': classes})

SEED = 900729
dir_path = './data'
train_path = './data/Training'
test_path = './data/Testing'
validation_path = './data/Validation'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Devic: {device}")

if os.path.exists(dir_path):
    print("Dane juz istnieją")
else:
    kaggle.api.authenticate()
    kaggle.api.dataset_download_files('masoudnickparvar/brain-tumor-mri-dataset', path='./data', unzip=True)
    tr_orginal_df = load(train_path)
    ts_df = load(test_path)
    val_df, tr_df = train_test_split(tr_orginal_df, train_size=0.25, random_state=SEED, stratify=tr_orginal_df['Class'])
    validation_path = './data/Validation'
    Path(validation_path).mkdir(exist_ok=True)

    classes = ['glioma', 'meningioma', 'notumor', 'pituitary']
    for class_name in classes:
        class_validation_path = Path(validation_path) / class_name
        class_validation_path.mkdir(exist_ok=True)

    errors = []

    for idx, row in val_df.iterrows():
        try:
            source_path = Path(row['Class Path'])

            filename = source_path.name
            class_name = row['Class']
            destination_path = Path(validation_path) / class_name / filename

            shutil.move(str(source_path), str(destination_path))

        except Exception as e:
            errors.append({
                'file': row['Class Path'],
                'class': row['Class'],
                'error': str(e)
            })


    if errors:
        print(f"Errors: {len(errors)}")
        for error in errors:
            print(f"  - {error['class']}: {Path(error['file']).name} - {error['error']}")

    val_df_updated = val_df.copy()
    val_df_updated['Class Path'] = val_df_updated.apply(
        lambda row: str(Path(validation_path) / row['Class'] / Path(row['Class Path']).name),
        axis=1
    )

    tr_df_updated = load(train_path)

    val_df = val_df_updated
    tr_df = tr_df_updated
    print("Pomyślnie pobrano dane")



Devic: cuda
Dane juz istnieją


# 🧠 ResNet – Residual Network (Resztkowa Sieć Neuronowa)

### Opis

#### 🔹 Dlaczego powstał?
- Wraz ze wzrostem liczby warstw w sieciach neuronowych pojawił się problem:
  - **zanikający gradient** – trudność w trenowaniu bardzo głębokich sieci,
  - **degradacja dokładności** – dodawanie kolejnych warstw pogarszało wyniki zamiast je poprawiać.
- Klasyczne sieci (np. VGG) nie były w stanie efektywnie uczyć się przy setkach warstw.

#### 🔹 I wtedy powstał ResNet
- ResNet wprowadza **połączenia resztkowe (skip connections)**.
- Zamiast uczyć się bezpośredniej funkcji `H(x)`, sieć uczy się **reszty** względem wejścia:

\[
H(x) = F(x) + x
\]

gdzie:
- `x` – wejście do bloku,
- `F(x)` – transformacja (np. sploty, normalizacja, ReLU),
- `H(x)` – wyjście po dodaniu shortcutu.

Dzięki temu sieć łatwo uczy się nawet **funkcji tożsamościowej**, co stabilizuje trening.

#### 🔹 Jak działa blok resztkowy?
Schemat działania jednego **Residual Block**:

Wejście (x)<br>
↓<br>
[Conv -> BN -> ReLU -> Conv -> BN] = F(x)<br>
↓<br>
Dodanie shortcut: y = F(x) + x<br>
↓<br>
ReLU<br>
↓<br>
Wyjście (y)

- **Shortcut (skip connection)** może być:
  - *Identity* – gdy rozmiary się zgadzają,
  - *Conv 1x1* – gdy trzeba dopasować liczbę kanałów lub rozdzielczość.


### Architektura

<p align="center">
  <img src="./screenshots/resnet_architecture.png" alt="Resnet architecture">
</p>

## Base line

In [9]:
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder(root=train_path, transform=data_transform)
test_dataset  = datasets.ImageFolder(root=test_path, transform=data_transform)
val_dataset   = datasets.ImageFolder(root=validation_path, transform=data_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [10]:
print(train_dataset)
print(val_dataset)
print(test_dataset)

Dataset ImageFolder
    Number of datapoints: 4284
    Root location: ./data/Training
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 1428
    Root location: ./data/Validation
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 1311
    Root location: ./data/Testing
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [11]:
model = resnet18(pretrained=True) 
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

model = model.to(device)

In [12]:
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")


AcceleratorError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
